# Exercice 2 - System evaluation

## Imports

In [127]:
import pandas as pd
import numpy as np
#import sklearn.metrics #import confusion_matrix

## Load data

Define the path of the data file

In [128]:
path = "ex2-system-a.csv"

Read the CSV file using `read_csv`

In [129]:
dataset_a = pd.read_csv(path, sep=";", index_col=False, names=["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "y_true"])

Display first rows

In [130]:
dataset_a.head()

,0,1,2,3,4,5,6,7,8,9,y_true
0,5.348450e-08,7.493480e-10,8.083470e-07,2.082290e-05,5.222360e-10,2.330260e-08,5.241270e-12,9.999650e-01,4.808590e-07,0.000013,7
1,1.334270e-03,3.202960e-05,8.504280e-01,1.669090e-03,1.546460e-07,2.412940e-04,1.448280e-01,1.122810e-11,1.456330e-03,0.000011,2
2,3.643050e-06,9.962760e-01,2.045910e-03,4.210530e-04,2.194020e-05,1.644130e-05,2.838160e-04,3.722960e-04,5.150120e-04,0.000044,1
3,9.998200e-01,2.550390e-10,1.112010e-05,1.653200e-05,5.375730e-10,8.999750e-05,9.380920e-06,4.464470e-05,2.418440e-06,0.000006,0
4,2.092460e-08,7.464220e-08,3.560820e-05,5.496200e-07,9.988960e-01,3.070920e-08,2.346150e-04,9.748010e-07,1.071610e-06,0.000831,4


Store some useful statistics (class names + number of classes)

In [131]:
class_names = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
nb_classes = len(class_names)

## Exercise's steps

a) Write a function to take classification decisions on such outputs according to Bayes’rule.

In [132]:
def bayes_classification(df,nb_classes,class_names):
        """
        Take classification decisions according to Bayes rule.

        Parameters
        ----------
        df : Pandas DataFrame of shape (n_samples, n_features + ground truth)
                Dataset.
                
        Returns
        -------
        preds : Numpy array of shape (n_samples,)
                Class labels for each data sample.
        """
        N =len(df)*nb_classes
        # give feature
        x = df[class_names].to_numpy()#[class_names]
        
        label = df['y_true']
        label = list(label)
        # create p(x) = probability of
        mu = np.mean(x)
        sigma = np.var(x)

        # Nn = len(x) -> y = n
        # create P_Cx = priori proba [10,0]
        P_Cx = [label.count(i)/len(label) for i in range(nb_classes)]
        # create P(x|Cx) = likelihood [10000 x 10]
        P_x_Cx = 1/np.sqrt((2*np.pi*sigma**2))*np.exp(-(x-mu)**2/(2*sigma**2))
        P_Cx_x = np.zeros((len(df)))
        evidence = sum([P_Cx[c] * P_x_Cx[c] for c in range(len(P_Cx))])
        for i in range(len(df)):
                #priori = P_Cx[i]#label.count(i)/N
                #likelihood = P_x_Cx[i]
    
                # create P(Cx|x) = posteriori 
                
                P_Cx_x[i] = np.argmax((P_x_Cx[i][:] * P_Cx)/evidence)
        y = P_Cx_x
        return y

b) What is the overall error rate of the system ?

In [133]:
# Your code here: compute and print the error rate of the system
N = nb_classes*len(dataset_a)
y_true = dataset_a['y_true']
y_pred = bayes_classification(dataset_a,nb_classes,class_names)
acc = np.sum(y_pred[(y_pred == y_true)])/N
err_rate = 1- acc
print(err_rate)

0.98389


c) Compute and report the confusion matrix of the system.

In [134]:
def confusion_matrix(y_true, y_pred, n_classes):
    """
    Compute the confusion matrix.
        
    Parameters
    ----------
    y_true : Numpy array of shape (n_samples,)
             Ground truth.
    y_pred : Numpy array of shape (n_samples,)
             Predictions.
    n_classes : Integer
                Number of classes.
         
    Returns
    -------
    cm : Numpy array of shape (n_classes, n_classes)
         Confusion matrix.
    """
    cm = pd.crosstab(y_true, y_pred)
    return cm

In [135]:
# Your code here: compute and print the confusion matrix
y_true = dataset_a['y_true']
y_pred = bayes_classification(dataset_a,nb_classes,class_names)
cm = confusion_matrix(y_true,y_pred,nb_classes)
print(cm)

col_0   0.0  1.0  2.0  3.0  4.0  5.0  6.0  7.0  8.0  9.0
y_true                                                  
0         9  863   36    8    4   26   12    2    9   11
1         1   13  107   18    3    7    4   24   37  921
2        13  865   33   40   11    4    9    8   27   22
3        20  731   50   46   10   71    4   15   29   34
4         1  716   21   11   23    4   14    8   10  174
5        42  600   18   62   33   19   16    5   55   42
6        24  741   88    6   31   24   24    0   13    7
7         4  789   29   25    5    0    0   24    4  148
8        21  601   61   59   36   38   17    4   32  105
9         7  624   10   38  107   13    2  122   15   71


d) What are the worst and best classes in terms of precision and recall ?

In [136]:
def precision_per_class(cm):
    """
    Compute the precision per class.
        
    Parameters
    ----------
    cm : Numpy array of shape (n_classes, n_classes)
         Confusion matrix.
         
    Returns
    -------
    precisions : Numpy array of shape (n_classes,)
                 Precision per class.
    """
    #cm = cm.to_numpy()
    Fp = cm.sum(axis=0) - np.diag(cm) 
    Tp = np.diag(cm)
    precisions = (Tp)/(Tp+Fp)
    return precisions

In [137]:
def recall_per_class(cm):
    """
    Compute the recall per class.
        
    Parameters
    ----------
    cm : Numpy array of shape (n_classes, n_classes)
         Confusion matrix.
         
    Returns
    -------
    recalls : Numpy array of shape (n_classes,)
              Recall per class.
    """
    Tp = np.diag(cm)
    Fn = cm.sum(axis=1) - np.diag(cm)
    recalls = Tp /(Tp+Fn)
    return recalls

In [138]:
# Your code here: find and print the worst and best classes in terms of precision
prec = precision_per_class(cm)
w_class = min(prec)
b_class = max(prec)
print(f"Worst class are {w_class} in label {prec.argmin()}")
print(f"Best class are {b_class} in label {prec.argmax()}")

Worst class are 0.0019868561821794286 in label 1
Best class are 0.23529411764705882 in label 6


In [139]:
# Your code here: find and print the worst and best classes in terms of recall
rec = recall_per_class(cm)
w_class = min(rec)
b_class = max(rec)
print(f"Worst class are {w_class} in label {rec.argmin()}")
print(f"Best class are {b_class} in label {rec.argmax()}")

Worst class are 0.009183673469387756 in label 0
Best class are 0.07036669970267592 in label 9


e) In file `ex1-system-b.csv` you find the output of a second system B. What is the best system between (a) and (b) in terms of error rate and F1.

In [140]:
# Your code here: load the data of the system B
path = "ex2-system-b.csv"
dataset_b = pd.read_csv(path, sep=";", index_col=False, names=["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "y_true"])
class_names = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
nb_classes = len(class_names)

In [141]:
# find confusion matrix for B
y_true_b = dataset_b['y_true']
y_pred_b = bayes_classification(dataset_b,nb_classes,class_names)
cm_b = confusion_matrix(y_true_b,y_pred_b,nb_classes)

In [142]:
def system_accuracy(cm):
    """
    Compute the system accuracy.
        
    Parameters
    ----------
    cm : Numpy array of shape (n_classes, n_classes)
         Confusion matrix.
         
    Returns
    -------
    accuracy : Float
               Accuracy of the system.
    """
    Tp = np.diag(cm)
    Fp = cm.sum(axis=0) - np.diag(cm) 
    Fn = cm.sum(axis=1) - np.diag(cm)
    Tn = cm.values.sum() - (Fp + Fn + Tp)
    N = np.sum(cm.to_numpy())
    accuracy = (Tp + Tn)/N
    return accuracy

In [143]:
def system_f1_score(cm):
    """
    Compute the system F1 score.
        
    Parameters
    ----------
    cm : Numpy array of shape (n_classes, n_classes)
         Confusion matrix.
         
    Returns
    -------
    f1_score : Float
               F1 score of the system.
    """
    prec = precision_per_class(cm)
    rec = recall_per_class(cm)

    f1_score = 2*(prec*rec)/(prec+rec)
    return f1_score

In [144]:
# Your code here: compute and print the accuracy and the F1 score of the system A
acc_a = system_accuracy(cm)
F1_a = system_f1_score(cm)
print(f"System A have a F1 of {F1_a.to_numpy()} and an accuracy of {acc_a.to_numpy()}\n")

System A have a F1 of [0.01604278 0.0033863  0.04444444 0.06953893 0.03694779 0.03460838
 0.04528302 0.03870968 0.05311203 0.05581761] and an accuracy of [0.8896 0.2348 0.8581 0.8769 0.8801 0.894  0.8988 0.8808 0.8859 0.7598]



In [145]:
# Your code here: compute and print the accuracy and the F1 score of the system B
acc_b = system_accuracy(cm_b)
F1_b = system_f1_score(cm_b)
print(f"System A have a F1 of {F1_b.to_numpy()} and an accuracy of {acc_b.to_numpy()}\n")

System A have a F1 of [0.00390244 0.00107216 0.02650177 0.01263538 0.03001876 0.0173913
 0.01       0.0270027  0.01964637 0.01569996] and an accuracy of [0.8979 0.0683 0.8898 0.8906 0.8966 0.9096 0.901  0.8919 0.9002 0.7743]

